# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import numpy as np
from functools import partial
import sys
import random
import pandas as pd
from typing import Tuple
sys.setrecursionlimit(10000)
import time
from tqdm import tqdm

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

class TreeDecision:

    class Node:
        def __init__(self, data, labels, perc_data, loss):
            self.data = data
            self.labels = labels
            self.perc_all_data = perc_data
            self.n_samples = data.shape[0]
            self.label_node, self.perc_label_node = self._compute_main_label()

            self.loss = loss(labels)

            self.sep_val = None
            self.sep_features = None
            self.left = None
            self.right = None

        def _compute_main_label(self,):
            label, counts = np.unique(self.labels, return_counts=True)
            main_label = np.argmax(counts)
            perc_main_labels = counts[main_label] / self.labels.shape[0]
            return int(label[main_label]), float(perc_main_labels)

    def __init__(self, criterion="entropy", max_depth=5, min_samples_split=5,
                 max_samples_leaf=5, purity=True, numbers_feature: Tuple[int] = None):
        """
        criterion: str
                  функция оптимизации
        max_depth: int
                  максимальной глубины дерева
        min_samples_split: int
                  минимального числа объектов в листе
        max_samples_leaf: int
                  максимальное количество листьев в дереве
        leaf_is_one_class: bool
                  остановка, если все объекты в листе относятся к одному классу
        """
        if criterion == "entropy":
            self.criterion = self._entropy
        elif criterion == "gini":
            self.criterion = self._gini
        else:
            raise NotImplementedError(f"criterion choise between Entropy and Gini \
       {criterion} don't implemented")

        self._max_depth = max_depth
        self._curr_msf = 0
        self._min_samples_split = min_samples_split
        self._max_samples_leaf = max_samples_leaf
        self._leaf_is_one_class = purity
        self._root = None
        self.n_maxs = []
        self.numbers_feature = numbers_feature

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        gini = 1 - sum(probabilities ** 2)
        return gini

    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))
        return entropy

    def fit(self, X, y):
        if self.numbers_feature is None:
            self.numbers_feature = tuple(range(X.shape[1]))
        self._root = self.Node(X, y, 1, loss=self.criterion)
        self._root = self._feed_tree(self._root, -1)
        return self

    @staticmethod
    def cutter(X, sep_val, n_features):
        # get left and right index for value equal optim loss
        left = np.argwhere(X[:, n_features] < sep_val)[:, 0]
        right = np.argwhere(X[:, n_features] >= sep_val)[:, 0]
        return left, right

    def _feed_tree(self, node, curr_depth=None):

        if node.data.shape[0] < self._min_samples_split or curr_depth == self._max_depth\
                or self._curr_msf == self._max_samples_leaf:
            if self._curr_msf != self._max_samples_leaf:
                self._curr_msf += 1
            return None
        curr_depth+=1

        # compute loss for each element
        _splitter = partial(self._splitter, y=node.labels,)
        losses = np.apply_along_axis(_splitter, 0, node.data.copy())[:, self.numbers_feature] # оставить только n-ые фичи

        # max_loss = np.max(losses)
        # for n_max in self.n_maxs:
        #     losses[n_max] = max_loss

        # end optimization node
        if np.max(losses) == 0 and self._leaf_is_one_class:
            self._curr_msf += 1
            return node

        # get optimal loss index
        min_ids = list(np.unravel_index(np.argmin(losses), losses.shape)) #n_samples, n_features
        min_ids[1] = self.numbers_feature[int(min_ids[1])]
        min_ids = tuple(min_ids)
        # надо резать не по индексу лоса а взять значения по индексу лоса и ним резать массив.

        node.sep_val = node.data[min_ids]
        node.sep_features = int(min_ids[1])
        left, right = self.cutter(node.data.copy(), node.sep_val, node.sep_features)
        perc_data = (left.shape[0] * node.perc_all_data) / node.data.shape[0]

        assert (left.shape[0] + right.shape[0]) == node.data.shape[0],\
            f"left {left.shape[0]} right {right.shape[0]}, all {node.data.shape[0]}"


        if left.shape[0] == 0:
            # выбиить разделяющее значение из последующего расчета лосса, иначе зацикливание
            node.right = self.Node(node.data[right, :], node.labels[right],
                                   node.perc_all_data - perc_data, loss=self.criterion)
            # self.n_maxs.append(losses[min_ids])
            node.right = self._feed_tree(node.right, curr_depth)
        elif right.shape[0] == 0:
            node.left = self.Node(node.data[left, :], node.labels[left],
                                  perc_data, loss=self.criterion)
            node.left = self._feed_tree(node.left, curr_depth)
        else:
            node.left = self.Node(node.data[left, :], node.labels[left],
                                  perc_data, loss=self.criterion)
            node.right = self.Node(node.data[right, :], node.labels[right],
                                   node.perc_all_data - perc_data, loss=self.criterion)
            node.left = self._feed_tree(node.left, curr_depth)
            node.right = self._feed_tree(node.right, curr_depth)
        return node

    def _splitter(self,X, y):
        # проблема с лоссом!

        def get_loss(split_val, X):
            left = np.argwhere(X < split_val)[:, 0]
            right = np.argwhere(X >= split_val)[:, 0]
            return (((left.shape[0] / y.shape[0]) * self.criterion(y[left])) + (
                    right.shape[0] / y.shape[0]) * self.criterion(y[right]))

        get_loss = partial(get_loss, X=X)
        return np.array(list(map(get_loss, X.copy())))

    def _detour_tree(self, node, X):
        if node.loss == 0:
            return node.label_node
        if X[node.sep_features] < node.sep_val:
            if node.left:
                label = self._detour_tree(node.left, X)
            else:
                return node.label_node
        else:
            if node.right:
                label = self._detour_tree(node.right, X)
            else:
                return node.label_node
        return label

    def predict(self, X):
        pred = np.zeros(X.shape[0])
        for idx in range(X.shape[0]):
            pred[idx] = self._detour_tree(self._root, X[idx, :])
        return pred

class RandomForest:

    def __init__(self, n_estimators=100, max_features="sqrt", criterion="entropy", max_depth=5, min_samples_split=5,
                 max_samples_leaf=5, purity=True, bs_size=0.8):
        """
        n_estimatorsint: int
                  число деревьев в ансамбле
        max_features: str
                тип вычисления максимального количества призанков
        criterion: str
                  функция оптимизации
        max_depth: int
                  максимальной глубины дерева
        min_samples_split: int
                  минимального числа объектов в листе
         max_samples_leaf: int
                  максимальное количество листьев в дереве
        leaf_is_one_class: bool
                  остановка, если все объекты в листе относятся к одному классу
        """
        self._n_estimators = n_estimators
        self._max_features = max_features
        self._criterion = criterion
        self._max_depth = max_depth
        self._min_samples_split = min_samples_split
        self._max_samples_leaf = max_samples_leaf
        self._purity = purity
        self.forest = None
        self._bs_size = bs_size

    def bootstrap(self,X,y, n_chunks,):
        assert X.shape[0] == y.shape[0]
        size_chunk = int(self._bs_size*X.shape[0])
        bs_data = []
        for _ in range(n_chunks):
            idxs = tuple([random.randint(0,X.shape[0] - 1) for _ in range(size_chunk)])
            bs_data.append((X[idxs, :].copy(), y[idxs,].reshape(-1).copy()))
        return bs_data

    def _get_n_features(self,n_features):
        if self._max_features == "sqrt":
            return tuple([random.randint(0, n_features - 1) for i in range(int(np.sqrt(n_features)))])

    def build_forest(self, n_features):
        self.forest = [TreeDecision(criterion=self._criterion, max_depth=self._max_depth,
                                    min_samples_split=self._min_samples_split, max_samples_leaf=self._max_samples_leaf,
                                    purity=self._purity, numbers_feature=self._get_n_features(n_features),)
                       for _ in range(self._n_estimators)]
        return True

    def fit(self,X, y):
        data = self.bootstrap(X, y, self._n_estimators)
        if self.build_forest(X.shape[1]):
            for idx in tqdm(range(len(self.forest))):
                self.forest[idx] = self.forest[idx].fit(data[idx][0], data[idx][1])
        return self

    @staticmethod
    def choose_preds(preds_line):
        values, counts = np.unique(preds_line, return_counts=True)
        finally_pred = values[np.argmax(counts)]
        return finally_pred

    def predict(self, X):
        preds = np.zeros((X.shape[0], self._n_estimators))
        for idx,tree in enumerate(self.forest):
            preds[:, idx] = tree.predict(X)
        pred_final = np.apply_along_axis(self.choose_preds, 1, preds)
        pred_all = preds
        return pred_final, pred_all

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
X = data["data"]
y = data["target"]


In [ ]:
model = TreeDecision(criterion="entropy", max_depth=100, min_samples_split=1,
                 max_samples_leaf=100, purity=True)
model.fit(X,y)
pred = model.predict(X)
print(f"My tree {accuracy_score(pred, y)}")


model = TreeDecision(criterion="gini", max_depth=100, min_samples_split=51,
                 max_samples_leaf=100, purity=True)
model.fit(X,y)
pred = model.predict(X)
print(f"My tree {accuracy_score(pred, y)}")

model = DecisionTreeClassifier()

model.fit(X,y)
pred = model.predict(X)
print(f"Box tree {accuracy_score(pred, y)}")

My tree 1.0
My tree 0.6666666666666666
Box tree 1.0


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
df = pd.read_csv("churn.csv").drop(columns=["RowNumber", "CustomerId"])

for ind, uniq_pay_system in enumerate(df.Surname.unique()):
    df.Surname = df.mask(df.Surname == uniq_pay_system, ind).Surname

for ind, uniq_pay_system in enumerate(df.Geography.unique()):
    df.Geography = df.mask(df.Geography == uniq_pay_system, ind).Geography

for ind, uniq_pay_system in enumerate(df.Gender.unique()):
    df.Gender = df.mask(df.Gender == uniq_pay_system, ind).Gender

X = df.drop(columns="Exited").to_numpy()
y = df["Exited"].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=41)

In [ ]:
obj = RandomForest(n_estimators=25, max_features="sqrt", criterion="entropy", max_depth=250, min_samples_split=10,
                 max_samples_leaf=100, purity=True, bs_size=0.01)

obj.fit(X_train, y_train)
predict,statistic = obj.predict(X_test)
print(np.unique(predict, return_counts=True))
print(f"My tree {accuracy_score(predict, y_test)}")

# obj = RandomForestClassifier(n_estimators=500, max_features="sqrt",  criterion="entropy", max_depth=100,min_samples_split=2,min_samples_leaf=1)
# obj.fit(X_train, y_train)
# predict = obj.predict(X_test)
# print(f"Top tree {accuracy_score(predict, y_test)}")

100%|██████████| 25/25 [00:53<00:00,  2.14s/it]


(array([0.]), array([2500]))
My tree 0.7948


In [ ]:
tr = np.where(statistic == y_test.reshape(-1,1),1, 0)

In [ ]:
features_tru = {idx:0 for idx in range(X_train.shape[1])}
features_tru

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}

In [ ]:
for pr in tr:
  for idx in range(pr.shape[0]):
    if pr[idx] == 1:
      for n_f in obj.forest[idx].numbers_feature:
        features_tru[n_f] += 1


In [ ]:
features_tru = {v:k for k,v in features_tru.items()}
features_tru

{0: 0,
 1: 3923,
 2: 7846,
 3: 9684,
 4: 1987,
 5: 8955,
 6: 7324,
 7: 5693,
 8: 10033,
 9: 0,
 10: 1987}

In [ ]:
sorted(features_tru.items())

[(0, 9),
 (1987, 10),
 (3923, 1),
 (5693, 7),
 (7324, 6),
 (7846, 2),
 (8955, 5),
 (9684, 3),
 (10033, 8)]

Можем сделать вывод что более важные признаки это 8 (баланс), 3 (пол),5 (время владения) 

In [ ]:
df

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,1,608,1,0,41,1,83807.86,1,0,1,112542.58,0
2,2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,4,850,1,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2539,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,606,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,905,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,1783,772,2,1,42,3,75075.31,2,1,0,92888.52,1
